In [1]:
import pandas as pd

In [2]:
senadores = pd.read_csv('BQ/senado/senadores_leg56.csv')
senadores = senadores[(senadores['legislatura'] == 56)&(senadores['ds_cargo']=='SENADOR')]
propo = pd.read_csv('BQ/senado/proposicoes.csv')
materia = pd.read_csv('BQ/senado/tramitações_senado.csv')

In [3]:
propo_filter = propo.merge(senadores, left_on='Codigo_Parlamentar', right_on='id_parlamentar')

In [4]:
len(propo_filter.id_parlamentar.unique())

55

In [5]:
len(senadores.id_parlamentar.unique())

55

In [6]:
propo_filter = propo_filter.merge(materia, left_on='Codigo_Materia', right_on='id', how='left')

In [7]:
propo_filter.columns

Index(['Codigo_Materia', 'Nome_Autor', 'Num_Order_Autor', 'Sigla_Tipo_Autor',
       'Codigo_Parlamentar', 'Nome_Parlamentar', 'Nome_Completo',
       'Sigla_Partido', 'Uf_Autor', 'Sexo', 'Forma_Tratamento', 'Email',
       'Url_Foto', 'Url_Site', 'id_parlamentar', 'nome_parlamentar',
       'nome_completo_parlamentar', 'uf', 'sigla_partido', 'legislatura',
       'ds_cargo', 'sq_candidato', 'sigla_partido_eleicao', 'sg_uf',
       'nome_candidato', 'nome_urna_candidato', 'cpf_candidato_str',
       'titulo_eleitoral_candidato_str', 'casa', 'id', 'descricao_tipo',
       'numero', 'ano', 'sequencia', 'f6_nome_orgao', 'descricao_situacao',
       'nome_autor', 'sigla_partido_antiga', 'sigla_partido_autor',
       'data_tramitacao_real'],
      dtype='object')

In [8]:
propo_filter = propo_filter[['Nome_Autor','descricao_tipo','ds_cargo']]

In [9]:
propo_filter.head()

,Nome_Autor,descricao_tipo,ds_cargo
0,Jorginho Mello,NaN,SENADOR
1,Jorginho Mello,REQUERIMENTO,SENADOR
2,Jorginho Mello,NaN,SENADOR
3,Jorginho Mello,NaN,SENADOR
4,Jorginho Mello,PROJETO DE LEI,SENADOR


In [10]:
propo_filter = propo_filter.groupby(['Nome_Autor','descricao_tipo'], as_index=False).size()

In [11]:
topicos_de_interesse = [
    'Projeto de Lei','Projeto de Lei Complementar', 'Projeto de Resolução', 'Proposta de Emenda à Constituição',
    'Projeto de Decreto Legislativo de Medida Provisória', 'Projeto de Decreto Legislativo de Sustação de Atos Normativos do Poder Executivo',
    'Requerimento de Audiência Pública', 'Requerimento de Convocação de Ministro de Estado no Plenário','Requerimento de Informação',
    'Indicação','Parecer do Relator','Emenda de Relator', 'Emenda de Plenário a Projeto com Urgência', 'Emenda de Plenário à MPV (Ato Conjunto 1/20)'
    ,'Emenda de Plenário', 'Emenda de Redação em Plenário', 'Emenda na Comissão', 'Emenda à PEC', 'Substitutivo'
                       ]

In [12]:
def get_mean_by_topic(df, topic):
    return (df[df.descricao_tipo==topic]['size'].sum()) / len(senadores.id_parlamentar.unique())

def get_median_by_topic(df, topic):
    return df[df.descricao_tipo==topic]['size'].median()

def get_df_parlamentar(df, nome):
    sen = df[df.Nome_Autor.str.contains(nome, regex=True)]
    sen['media_senado'] = sen.descricao_tipo.apply(lambda x: get_mean_by_topic(df, x))
    sen['mediana_senado'] = sen.descricao_tipo.apply(lambda x: get_median_by_topic(df, x))
    return sen

In [16]:
propo_filter.descricao_tipo.unique()

array(['PROJETO DE DECRETO LEGISLATIVO', 'PROJETO DE LEI',
       'PROJETO DE LEI COMPLEMENTAR', 'PROPOSTA DE EMENDA À CONSTITUIÇÃO',
       'RECURSO (SF)', 'REQUERIMENTO', 'REQUERIMENTO (CN)',
       'PROJETO DE RESOLUÇÃO DO SENADO', 'INDICAÇÃO',
       'PROPOSTA DE FISCALIZAÇÃO E CONTROLE',
       'PROJETO DE RESOLUÇÃO DO CONGRESSO NACIONAL'], dtype=object)

In [13]:
alessandro = get_df_parlamentar(propo_filter, 'Alessandro')

/tmp/ipykernel_30335/4173387890.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sen['media_senado'] = sen.descricao_tipo.apply(lambda x: get_mean_by_topic(df, x))
/tmp/ipykernel_30335/4173387890.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sen['mediana_senado'] = sen.descricao_tipo.apply(lambda x: get_median_by_topic(df, x))


In [14]:
def save_df(df, proponente=False):
    nome = '_'.join(df.Nome_Autor.iloc[0].split(' '))
    df['media_senado'] = df['media_senado'].round()
    df['mediana_senado'] = df['mediana_senado'].round()
    if proponente:
        df.to_excel(f'resultados/Senado/{nome}_proponentes.xlsx')
        return
    df.to_excel(f'resultados/Senado/{nome}_autores.xlsx')

In [15]:
save_df(alessandro)

/tmp/ipykernel_30335/253322932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['media_senado'] = df['media_senado'].round()
/tmp/ipykernel_30335/253322932.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mediana_senado'] = df['mediana_senado'].round()
